##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Temel Gerilim: Yakit verimliliginin tahmini

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/regression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

*Gerilim* problemlerinde ucret ya da olasilik gibi devamli degerlerin ciktisini tahmin etmeyi amaclariz. *Siniflandirma* problemlerinde ise sinif listesinden bir tanesini secmeyi hedefleriz (ornegin, resimdekini meyveyi taniyarak, onun elma mi portakal mi oldugunu soylemek).

Bu kitapciktak klasik [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) veri setini kullanarak 1970 sonu ve 1980 basindaki arabalarda yakit verimliligini tahmin eden bir model olusturacagiz. Bunu yapabilmek icin modelimize soz konusu zaman dilimindeki arabalarin ozelliklerini verecegiz. Bu ozellikler icinde arabanin kac beygir oldugu, agirligi, silindirleri ve yer degistirmesi bulunur.

Bu ornekte `tf.keras` APIsini kullanacagiz. Detayli bilgi icin [buraya bakiniz](https://www.tensorflow.org/guide/keras).

In [0]:
# Pairplot icin seaborn kullanalim
!pip install seaborn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version sadece Colab icinde bulunur.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

## Auto MPG veri setit

Bu veri setini [UCI Makine Ogrenmesi Havuzunda](https://archive.ics.uci.edu/ml/) bulabilirsiniz.



### Veriyi Alalim
Oncelikle veri setini indirelim.

In [0]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

Pandas kullanarak iceri aktaralim

In [0]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

### Veriyi Temizleyelim

Bu veri seti birkac adet bilinmeyen deger icerir.

In [0]:
dataset.isna().sum()

Bu ilk ornegi basit tutmak icin bu degerlerin oldugu siralari cikartalim.

In [0]:
dataset = dataset.dropna()

`"Origin"` sutunu sayisal degil, kategoriseldir. Bu yuzden onu one-hot'a donusturmemiz gerekir:

In [0]:
origin = dataset.pop('Origin')

In [0]:
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

### Veriyi Egitim ve Test Setlerine Bol

Simdi veri setini egitim seti ve test seti olarak ikiye ayiralim.

Test setini modelimizin degerlendirilmesinde kullanacagiz.

In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

### Veriyi Inceleyelim

Egitim setinden birkac sutunun birlesik dagilimina bir goz atalim.

In [0]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")

Ayni zamanda tam istatistiklere de bakalim:

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

### Ozellikleri Etiketlerden Ayirin

Hedef degeri ya da "etiketi" ozelliklerden ayiralim. Bu etiket modelin egitimden sonra tahmin etmesi gereken deger.

In [0]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

### Veriyi normallestirin

Yukaridaki `train_stats` bloguna tekrar bakin ve ozelliklerin degerlerinin ne kadar farkli oldugunu gozlemleyin.

Farkli olcekteki ve araliklardaki ozellikleri normallestirmek iyi bir uygulamadir. Modelimiz normallestirme olmadan da sonuca *ulasabilir*, fakat bu daha zordur ve elde ettigimiz model girdide belirtilen unitelere bagli kalmak zorunda kalir.

Not: Bu istatistikleri egitim setinden elde ettik ama test setini normallestirirken de ayni degerleri kullanacagiz. Test veri setini modelin egitildigi dagilima yansitmak icin bunu yapmaliyiz.

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

Bu normallestirilmis veriyi modelin egitilmesinde kullanacagiz.

Dikkat: Buradaki girdileri normallestirirken kullandimiz istatistikler (ortalama ve standart sapma) ve one-hot kodlama modele verilen butun verilere uygulanmalidir. Buna test veri seti ve uretimde kullanilan modele verilen canli veri de dahildir.

## Model

### Modeli olusturun

Modelimizi olusturalim. Burada iki yogun bagli gizli tabakali ve tek bir devamli deger donduren cikti tabakali `Sequential` modelini kullanacagiz. Daha sonra ikinci bir model daha olusturacagimiz icin modeli olusturma basamaklari `build_model` fonksiyonu icine alinmistir.

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [0]:
model = build_model()

### Modeli Inceleyin

Modelin basit bir aciklamasini elde etmek icin `.summary` yontemini kullanin

In [0]:
model.summary()


Simdi modeli deneyelim. Egitim setinden `10` orneklik bir set alalim ve onlarla `model.predict` yontemini cagiralim.

In [0]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

Beklenen sekli ve turu ureterek calisiyor gibi gozukuyor.

### Modeli Egitin

Modelimizi 1000 devir boyunca egitelim ve `history` nesnesine egitim ve dogrulama degerlerini kaydedelim.

In [0]:
# Biten her devir icin bir nokta yazdirarak egitim asamalarini gosterin
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

Modelin egitim asamalarini `history` nesnesinde saklanan istatistikleri kullanarak gosterebiliriz.

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

Bu grafikte, yaklasik 100 devir sonrasinda bir ilerleme ya da dogrulama hatasinda bir dusus goruyoruz. Simdi dogrulama skoru ilerleme gostermeyi birakinca egitimi otomatik olarak durduracak sekilde `model.fit` cagrisini guncelleyelim. Her devir icin egitim sartini test eden bir *EarlyStopping geri cagirimi* kullanacagiz. Eger belli sayida devir sonunda iyilesme gorulmezse egitim otomatik olarak durdurulur.

Bu geri cagirim hakkinda daha cok bilgiyi [burada](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping) bulabilirsiniz.

In [0]:
model = build_model()

# Iyilesme icin kontrol edilecek devir sayisi patience degistirgesi ile belirlenir
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

Grafikte gordumuz gibi dogrulama seti ortalama hata payini +/- 2 MPG civarinda gostermis. Bu iyi mi? Bu karari size birakiyoruz.

Modeli **test** setinde nasil genelledigine bakalim. Bu modelin gercek hayatta nasil tahminlerde bulunacagi hakkinda bilgi verir.

In [0]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

### Tahminde Bulunun

En sonunda test setindeki veriyi kullanarak MPG degerlerini tahmin edebiliriz:

In [0]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])


Gorunuse gore modelimiz oldukca iyi tahminlerde bulunuyor. Simdi de hata dagilimina bakalim.

In [0]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

Ornek sayimiz cok az oldugu icin tam bir normal dagilim goremedik.

## Sonuc

Bu kitapcikta gerilim problemini cozerken kullanabilecegimiz birkac teknik gorduk.

* Mean Squared Error (MSE) gerilim problemlerinde kullanilan bir kayip fonksiyonudur (siniflandirma probleminda farkli kayip fonksiyonlari kullanmistik).
* Degerlendirme olculeri de siniflandirma probleminden farklidir. Genel gerilim olcusu Mean Absolute Error (MAE) idir.
* Sayisal girdi ozellikleri farkli araliklar gosteriyorsa, bu ozellikler birbirinden bagimsiz olarak ayni araliga indirgenmelidir.
* Asiri uyumu engellemek icin elimizde yeteri kadar egitim verisi yoksa, az sayida gizli tabakali kucuk bir ag kullanmak tercih edilir.
* Erken durma da asiri uyumu engelleyen kullanisli bir tekniktir.